In [42]:
import pandas as pd
import yfinance as yf
from functools import reduce
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe

pd.set_option('display.max_columns', 1500)
pd.set_option('display.max_row', 1500)

# Extracting data  from Wiki & yfinance. In total is taking around 10 min

In [43]:
#Extracting actual companies in S&P 500 Index + calculating lenght of companies in the index
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = table[0]
sp500 = sp500["Symbol"].tolist()
sp500_len = len(sp500)

#Table from which will merge coulumns into main table - adding more info about S&P 500
aditional_info_sp500=table[0]
aditional_info_sp500=aditional_info_sp500[["Symbol","Founded","GICS Sector","GICS Sub-Industry","Date added"]]

In [44]:
#For loop whcih to store data for S&P 500 companies from yfinance to DataFrame in Pandas 
#IT IS LOADING APPROXIMATELY 5 MINUTES
list_stocks=[]
for i in range(0,sp500_len):
    stock = yf.Ticker(sp500[i])
    stock_i=stock.info
    stock_i=pd.DataFrame.from_dict(stock_i.items())
    stock_i.columns =['Name', 'Value'] 
    stock_i.rename(columns = {'Value':sp500[i]}, inplace = True)
    list_stocks.append(stock_i)

KeyboardInterrupt: 

In [ ]:
#Adding more infro
#IT IS LOADING APPROXIMATELY 5 MINUTES
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
comp_officers = []
for x in sp500["Symbol"][0:sp500_len]:  
    stock = yf.Ticker(x)
    i = stock.info.get('companyOfficers', [{}])[0]
    comp_officers.append({**i, 'Symbol': x})

officers = pd.DataFrame(comp_officers)
officers=officers[['Symbol','name','age','title','totalPay',]]
officers=pd.merge(officers, aditional_info_sp500, on='Symbol', how='outer', indicator=True)
officers=officers.drop(columns=['_merge']).rename(columns={"Symbol": "symbol"})

# Formating of the DataFrame

In [ ]:
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Name'], how='outer'), list_stocks)

final_df = final_df.reset_index(drop=True)
final_df=final_df.T
final_df.columns=final_df.iloc[0]
final_df=final_df.iloc[1:, :]

#Adding data from officers
final_df=pd.merge(final_df, officers, on='symbol', how='outer', indicator=True)

#Remove day and month from Date added column
final_df['Date added'] = final_df['Date added'].str[:4]
#final_df['Founded'] = final_df['Founded'].str[5:]

#Creating new Columns in main datafram
final_df['ND_to_EBITDA']=(final_df['totalDebt']-final_df['totalCash'])/final_df['ebitda'].astype(float)
final_df['Net_Debt']=(final_df['totalDebt']-final_df['totalCash']).astype(float)
final_df['PCT_Below_Low']=(final_df['previousClose']-final_df['targetLowPrice'])/final_df['targetLowPrice'].astype(float)
final_df['PCT_to_Low']=(final_df['targetLowPrice']-final_df['previousClose'])/final_df['previousClose'].astype(float)
final_df['PCT_to_Mean']=(final_df['targetMeanPrice']-final_df['previousClose'])/final_df['previousClose'].astype(float)
final_df['Date']=pd.Timestamp.today()   

#SETTING NEW ORDER OF COLUMNS in dataset + excluding columns which are not important 
new_columns_order = [
    "shortName","symbol","marketCap","enterpriseValue","previousClose","totalRevenue","ebitda","trailingPE",
    "forwardPE","returnOnEquity","returnOnAssets","fiftyDayAverage","twoHundredDayAverage",
    "recommendationMean","recommendationKey","sector","industry","fiftyTwoWeekLow","fiftyTwoWeekHigh",
    "trailingEps","forwardEps","targetHighPrice","targetLowPrice","targetMeanPrice","targetMedianPrice",
    "numberOfAnalystOpinions","totalCash","volume","totalDebt","quickRatio","currentRatio","beta",
    "debtToEquity","regularMarketVolume","averageVolume","payoutRatio","52WeekChange","bookValue",
    "priceToBook","fullTimeEmployees","state","country","website","longBusinessSummary","auditRisk",
    "boardRisk","compensationRisk","shareHolderRightsRisk","overallRisk","dividendRate","dividendYield",
    "exDividendDate","fiveYearAvgDividendYield","priceToSalesTrailing12Months","trailingAnnualDividendRate",
    "trailingAnnualDividendYield","profitMargins","floatShares","sharesOutstanding","sharesShort",
    "shortRatio","shortPercentOfFloat","lastFiscalYearEnd","nextFiscalYearEnd","mostRecentQuarter",
    "earningsQuarterlyGrowth","netIncomeToCommon","pegRatio","lastSplitFactor","lastSplitDate","enterpriseToRevenue",
    "enterpriseToEbitda","lastDividendValue","lastDividendDate","grossProfits","freeCashflow",
    "operatingCashflow","earningsGrowth","revenueGrowth","grossMargins","ebitdaMargins","operatingMargins",
    "trailingPegRatio" ,"open", "dayLow","dayHigh", "name" , "Date added", "age", "title", "totalPay", "Founded",
    "GICS Sector", "GICS Sub-Industry", "ND_to_EBITDA", "PCT_Below_Low", "PCT_to_Low", "PCT_to_Mean","Net_Debt"
            ]
final_df=final_df[new_columns_order]

#Google has 2 type of stocks - GOOG and GOOGL, removing GOOGL,
final_df = final_df.loc[final_df["symbol"] != "GOOGL"]

#Aproximately 2 stocks from S&P do not contain data, remove them. 
final_df = final_df[final_df['shortName'].notna()] 

#Fixxing date format
final_df['lastFiscalYearEnd'] = pd.to_datetime(final_df['lastFiscalYearEnd'], unit='s')
final_df['mostRecentQuarter'] = pd.to_datetime(final_df['mostRecentQuarter'], unit='s')
final_df['exDividendDate'] = pd.to_datetime(final_df['exDividendDate'], unit='s')
final_df['nextFiscalYearEnd'] = pd.to_datetime(final_df['nextFiscalYearEnd'], unit='s')
final_df['lastSplitDate'] = pd.to_datetime(final_df['lastSplitDate'], unit='s')
final_df['lastDividendDate'] = pd.to_datetime(final_df['lastDividendDate'], unit='s')

#Formatting to Billions, Milions, Thousands
final_df[["marketCap","enterpriseValue","ebitda","totalRevenue","totalCash","totalDebt","freeCashflow","operatingCashflow","grossProfits"]] = final_df[[
          "marketCap","enterpriseValue","ebitda","totalRevenue","totalCash","totalDebt","freeCashflow","operatingCashflow",
          "grossProfits"]]/1000000000

final_df[["floatShares","sharesOutstanding","sharesShort"]] = final_df[["floatShares","sharesOutstanding","sharesShort"]]/1000000

final_df[["volume","averageVolume","regularMarketVolume","totalPay"]] = final_df[["volume","averageVolume","regularMarketVolume","totalPay"]]/1000000
final_df["fullTimeEmployees"] = final_df["fullTimeEmployees"]/1000
final_df[["52WeekChange","fiveYearAvgDividendYield"]] = final_df[["52WeekChange","fiveYearAvgDividendYield"]]/100

#Transfering from object to number
final_df[['recommendationMean','returnOnEquity', 'returnOnAssets' , 'fiveYearAvgDividendYield','trailingAnnualDividendYield','52WeekChange']] = final_df[[
          'recommendationMean','returnOnEquity', 'returnOnAssets' , 'fiveYearAvgDividendYield','trailingAnnualDividendYield','52WeekChange']].astype(float)

##Round .2
final_df[['ND_to_EBITDA','totalCash','dividendYield','beta','fullTimeEmployees','fiftyDayAverage','twoHundredDayAverage','enterpriseToEbitda']] = final_df[[
          'ND_to_EBITDA','totalCash', 'dividendYield','beta','fullTimeEmployees','fiftyDayAverage','twoHundredDayAverage','enterpriseToEbitda']].astype(float).round(2)
#Round .1
final_df[['totalPay','forwardPE']] = final_df[['totalPay','forwardPE']].astype(float).round(1)

#Transfer to intiger
final_df[['numberOfAnalystOpinions','bookValue','priceToBook','enterpriseToRevenue','floatShares','sharesOutstanding','sharesShort','grossProfits','fullTimeEmployees','totalCash', 'twoHundredDayAverage','fiftyDayAverage','regularMarketVolume','averageVolume','marketCap','enterpriseValue','previousClose','totalRevenue','ebitda','volume','totalDebt','debtToEquity','freeCashflow','operatingCashflow']] = final_df[[
          'numberOfAnalystOpinions','bookValue','priceToBook','enterpriseToRevenue','floatShares','sharesOutstanding','sharesShort','grossProfits','fullTimeEmployees','totalCash','twoHundredDayAverage','fiftyDayAverage','regularMarketVolume','averageVolume','marketCap','enterpriseValue','previousClose','totalRevenue','ebitda','volume','totalDebt','debtToEquity','freeCashflow','operatingCashflow']].fillna(0).astype(int)

final_df=final_df.sort_values(by=['marketCap'], ascending=False)

#CHANGING NAMES OF THE COLUMNS
final_df.rename(columns = 
        {'lastDividendDate':'Last_Div_Date','lastDividendValue':'Last_Div_Value',
        'grossProfits':'Gross_Profits','revenueGrowth':'Rev_Gr','trailingPegRatio':'T_Peg_Ratio',
        'grossMargins':'Gross_Mgn','ebitdaMargins':'EBITDA_Mgn','grossMargins':'Gross_Mgn',
        'operatingMargins':'Operating_Mgn','earningsGrowth':'Earn_Gr','earningsGrowth':'Earnings_Growth',
        'LastDividendDate':'Last_Dividend_Date','symbol':'Symbol','enterpriseToEbitda':'Ent_to_EBITDA','enterpriseToRevenue':'Ent_to_Rev',
        'lastSplitDate':'Last_Split_Date','lastSplitFactor':'Last_Split_Factor','pegRatio':'PEG_Ratio',
        'netIncomeToCommon':'Net_Income_To_Common','earningsQuarterlyGrowth':'Earnings_Quarterly_Growth',
        'mostRecentQuarter':'Most_Recent_Quarter','nextFiscalYearEnd':'Next_Fiscal_Year_End',
        'lastFiscalYearEnd':'Last_Fiscal_Year_End','Last_Fiscal_Year_End':'Last_Fiscal_Year_End',
        'shortPercentOfFloat':'Short_Pct_Of_Float','shortRatio':'Short_Ratio','sharesShort':'Shares_Short',
        'sharesOutstanding':'Shares_Outst','floatShares':'Float_Shares','profitMargins':'Profit_Mgn',
        'currency':'Cur','trailingAnnualDividendYield':'T_An_Div_Y','trailingAnnualDividendRate':'T_An_Div_Rate',
        'priceToSalesTrailing12Months':'PS_T_12M','fiveYearAvgDividendYield':'5Y_Avg_Div_Y',
        'exDividendDate':'Ex_Div_Date','dividendYield':'Div_Y','dividendRate':'Div_Rate',
        'overallRisk':'Overall_Risk','shareHolderRightsRisk':'SH_Rights_Risk','compensationRisk':'Comp_Risk',
        'boardRisk':'Board_Risk','auditRisk':'Audit_Risk','longBusinessSummary':'Business_Summary','website':'Web_Site',
        'sector':'Sector','country':'Country','state':'State', 'priceToBook':'Price_to_Book','bookValue':'Book_Value', 
        'averageVolume':'Avg_Vol','payoutRatio':'Payout_Ratio','regularMarketVolume':'Reg_Vol','debtToEquity':'D_to_E',
        '52WeekChange':'52W_Change','marketCap':'Market_Cap','fullTimeEmployees':'Empl_K','recommendationMean':'Rec_Mean',
        'freeCashflow':'FCF','operatingCashflow':'OCF','recommendationKey':'Rec_Key','shortName':'Company_Name','previousClose':'Close', 
        'trailingPE':'T_PE', 'forwardPE':'F_PE', 'returnOnEquity':'ROE', 'returnOnAssets':'ROA', 'enterpriseValue':'Ent_Value', 
        'fiftyDayAverage':'50D_Avg','twoHundredDayAverage':'200D_Avg', 'industry':'Industry', 'industryDisp':'Industry_Disp', 
        'fiftyTwoWeekLow':'52W_Low', 'fiftyTwoWeekHigh':'52W_High', 'trailingEps':'T_EPS', 'forwardEps':'F_EPS', 'ebitda':'EBITDA', 
        'totalRevenue':'Rev', 'targetHighPrice':'Target_HP', 'targetLowPrice':'Target_LP', 'targetMeanPrice':'Target_Mean_P', 
        'targetMedianPrice':'Target_Median_P', 'numberOfAnalystOpinions':'Anlsts', 'totalCash':'Total_Cash', 'volume':'Vol_M', 
        'totalDebt':'Total_Debt', 'quickRatio':'Quick_Ratio', 'currentRatio':'Current_Ratio', 'beta':'Beta', 'open':'Open',
        'dayLow':'Day_Low','dayHigh':'Day_High', 'name':'CEO_Name', 'age': 'CEO_Age', 'title':'CEO_Title', 'totalPay':'CEO_Salary', 'Date added':'Added'}, inplace = True)


# Final result - table with data for S&P 500

In [ ]:
final_df.sample(5)

,Company_Name,Symbol,Market_Cap,Ent_Value,Close,Rev,EBITDA,T_PE,F_PE,ROE,ROA,50D_Avg,200D_Avg,Rec_Mean,Rec_Key,Sector,Industry,52W_Low,52W_High,T_EPS,F_EPS,Target_HP,Target_LP,Target_Mean_P,Target_Median_P,Anlsts,Total_Cash,Vol_M,Total_Debt,Quick_Ratio,Current_Ratio,Beta,D_to_E,Reg_Vol,Avg_Vol,Payout_Ratio,52W_Change,Book_Value,Price_to_Book,Empl_K,State,Country,Web_Site,Business_Summary,Audit_Risk,Board_Risk,Comp_Risk,SH_Rights_Risk,Overall_Risk,Div_Rate,Div_Y,Ex_Div_Date,5Y_Avg_Div_Y,PS_T_12M,T_An_Div_Rate,T_An_Div_Y,Profit_Mgn,Float_Shares,Shares_Outst,Shares_Short,Short_Ratio,Short_Pct_Of_Float,Last_Fiscal_Year_End,Next_Fiscal_Year_End,Most_Recent_Quarter,Earnings_Quarterly_Growth,Net_Income_To_Common,PEG_Ratio,Last_Split_Factor,Last_Split_Date,Ent_to_Rev,Ent_to_EBITDA,Last_Div_Value,Last_Div_Date,Gross_Profits,FCF,OCF,Earnings_Growth,Rev_Gr,Gross_Mgn,EBITDA_Mgn,Operating_Mgn,T_Peg_Ratio,Open,Day_Low,Day_High,CEO_Name,Added,CEO_Age,CEO_Title,CEO_Salary,Founded,GICS Sector,GICS Sub-Industry,ND_to_EBITDA,PCT_Below_Low,PCT_to_Low,PCT_to_Mean,Net_Debt
329,"Motorola Solutions, Inc.",MSI,51,57,315,9,2,31.363638,24.5,NaN,0.12640,302,287,1.8,buy,Technology,Communication Equipment,251.0,329.83,9.9,12.66,370.0,320.0,338.76,335.0,6,0,2,6,0.701,0.952,0.94,1729,2,0,0.3559,0.002405,2,142,20,IL,United States,https://www.motorolasolutions.com,"Motorola Solutions, Inc. provides public safet...",4,10,2,1,6,3.92,0.01,2023-12-14,0.0138,5.23923,3.52,0.011146,0.17304,165,165,1,1.67,0.0087,2022-12-31,2023-12-31,2023-09-30,0.663,1702000000,3.75,10000:40425,2011-01-04,5,20.31,0.98,2023-12-14,4,1,2,0.656,0.077,0.49542,0.28599,0.26252,2.1061,310.9,308.2,315.05,Mr. Gregory Q. Brown,1957,62.0,Chairman & CEO,4.9,1928 (2011),Information Technology,Communications Equipment,1.99,-0.013063,0.013235,0.072636,5.593000e+09
60,"Bath & Body Works, Inc.",BBWI,9,14,41,7,1,13.133758,12.5,NaN,0.14878,32,35,2.4,buy,Consumer Cyclical,Specialty Retail,27.3,49.55,3.14,3.3,60.0,30.0,40.28,38.5,18,0,3,5,0.432,1.39,1.81,0,3,3,0.2548,0.000340,-9,0,8,OH,United States,https://www.bbwinc.com,"Bath & Body Works, Inc. operates a specialty r...",8,10,5,4,8,0.8,0.02,2023-11-16,0.0547,1.258313,0.8,0.019180,0.09899,210,225,10,2.93,0.0537,2023-01-28,2024-01-28,2023-10-28,0.308,727000000,3.24,1237:1000,2021-08-03,1,9.79,0.2,2023-11-16,3,0,1,0.303,-0.026,0.42512,0.20162,0.14149,1.3832,41.88,40.92,41.955,Ms. Gina R. Boswell,1983,59.0,CEO & Director,3.1,1963,Consumer Discretionary,Other Specialty Retail,3.55,0.390333,-0.280748,-0.034284,5.297000e+09
198,FirstEnergy Corp.,FE,21,46,37,12,3,38.926315,13.9,0.05590,0.02739,36,37,2.5,buy,Utilities,Utilities - Regulated Electric,32.18,43.31,0.95,2.66,44.0,32.0,39.5,40.0,14,0,8,24,0.353,0.525,0.47,223,8,3,1.6421,-0.000766,18,2,12,OH,United States,https://www.firstenergycorp.com,"FirstEnergy Corp., through its subsidiaries, g...",6,6,3,3,4,1.64,0.04,2023-11-06,0.0400,1.669526,1.56,0.041194,0.04123,514,573,11,3.62,0.0227,2022-12-31,2023-12-31,2023-09-30,0.198,545000000,2.28,NaN,NaT,3,12.94,0.41,2023-11-06,8,-1,1,0.196,0.003,0.65987,0.27978,0.20505,2.4178,37.7,36.8,37.73,Mr. K. Jon Taylor,1997,48.0,Senior VP of Strategy & CFO,1.5,1997,Utilities,Electric Utilities,6.84,0.183437,-0.155004,0.043042,2.433600e+10
426,"Steel Dynamics, Inc.",STLD,20,20,119,19,3,7.936265,12.1,0.31909,0.14631,109,106,3.0,hold,Basic Materials,Steel,90.55,136.46,15.69,10.31,130.0,73.0,109.22,108.0,9,2,6,3,2.162,3.768,1.48,34,6,1,0.1029,0.001732,54,2,12,IN,United States,https://stld.steeldynamics.com,"Steel Dynamics, Inc., together with its subsid...",3,8,5,3,4,1.7,0.01,2023-12-28,0.0219,1.039258,1.615,0.013557,0.13727,151,161,6,5.13,0.0466,2022-12-31,2023-12-31,2023-09-30,-0.369,2661476096,-0.52,2:1,2008-03-31,1,5.48,0.425,2023-09-28,6,1,3,-0.31,-0.188,0.22192,0.19703,0.15992,None,120.2,119.185,127.76,Mr. Mark D. Millett,2022,62.0,"Chairman, Co-Founder & CEO",6.6,1993,Materials,Steel,0.22,0.631918,-0.387224,-0.083186,8.287601e+08
261,"IQVIA Holdings, Inc.",IQV,41,53,230,14,2

In [ ]:
%store final_df

Stored 'final_df' (DataFrame)


In [ ]:
GSHEET_NAME = 'SP 500 + Good Opportunites'
TAB_NAME = 'SP500' 

def write_df_to_sheet(GSHEET_NAME, TAB_NAME, df):
    gc= gspread.service_account(filename ='sp500opp-81786021999a.json')
    sh=gc.open(GSHEET_NAME)
    worksheet = sh.worksheet(TAB_NAME)
    set_with_dataframe(worksheet, df)

write_df_to_sheet(GSHEET_NAME, TAB_NAME, final_df)   

# Good opportunites for investment based on filters by Analysts Reccomendation and Beta

In [52]:
good_opportunities=final_df[
    (final_df['Rec_Key'] == "buy") & (final_df['Rec_Mean'] <= 1.9) & (final_df['Market_Cap'] > 10) & (final_df['Beta'] < 20)]
good_opportunities = good_opportunities.fillna('-')
good_opportunities = pd.pivot_table(good_opportunities, index=[
    'Sector','Industry','Symbol','Company_Name','Founded','Added','Close',"Target_LP","PCT_to_Low","Target_Mean_P",
    'PCT_to_Mean',"Target_HP",'Anlsts','Beta', 'Vol_M','52W_Change','ND_to_EBITDA','D_to_E','OCF','FCF',"Rec_Mean",
    "T_PE","F_PE","Ent_to_EBITDA","Div_Y","Overall_Risk", "Earnings_Growth","EBITDA_Mgn","Operating_Mgn",
    "CEO_Salary","Ent_Value",], values="Market_Cap").reset_index()
#good_opportunities= good_opportunities.style.format({ ('PCT_to_Low'):'{:.2%}}',('PCT_to_Mean'):'{0:,.0%}',('52W_Change'):'{0:,.0%}',('EBITDA_Mgn'):'{0:,.0%}',('Operating_Mgn'):'{0:,.0%}',('Target_Mean_P'):'{0:,.0f}',('Target_LP'):'{0:,.0f}',('Target_HP'):'{0:,.0f}',('Beta'):'{0:,.1f}',('CEO_Salary'):'{0:,.1f}',('Rec_Mean'):'{0:,.1f}',('T_PE'):'{0:,.0f}',('F_PE'):'{0:,.0f}'})
#good_opportunities = good_opportunities.style.format({'PCT_to_Low': '{:.2%}', 'PCT_to_Mean': '{:.2%}', '52W_Change': '{:.2%}'})

good_opportunities

,Sector,Industry,Symbol,Company_Name,Founded,Added,Close,Target_LP,PCT_to_Low,Target_Mean_P,PCT_to_Mean,Target_HP,Anlsts,Beta,Vol_M,52W_Change,ND_to_EBITDA,D_to_E,OCF,FCF,Rec_Mean,T_PE,F_PE,Ent_to_EBITDA,Div_Y,Overall_Risk,Earnings_Growth,EBITDA_Mgn,Operating_Mgn,CEO_Salary,Ent_Value,Market_Cap
0,Basic Materials,Building Materials,VMC,Vulcan Materials Company (Holdi,1909,1999,224,215.00,-0.043424,244.70,0.088717,264.0,22,0.79,1,0.002813,2.18,60,1,0,1.8,35.838657,27.8,17.92,0.01,6,0.566,0.24694,0.20235,4.3,33,29
1,Communication Services,Entertainment,LYV,"Live Nation Entertainment, Inc.",2010,2019,93,85.00,-0.086709,111.17,0.194477,128.0,18,1.28,6,0.003609,1.55,495,1,1,1.9,62.0,41.0,15.82,-,10,0.281,0.07508,0.07568,22.3,25,21
2,Communication Services,Internet Content & Information,GOOG,Alphabet Inc.,1998,2006,133,130.00,-0.024024,146.10,0.096847,160.0,10,1.03,58,0.004941,-0.93,11,106,70,1.7,25.639847,20.2,16.5,-,10,0.46,0.32333,0.27962,7.9,1585,1666
3,Communication Services,Internet Content & Information,META,"Meta Platforms, Inc.",2004,2013,333,175.00,-0.474743,372.82,0.119008,435.0,50,1.20,31,0.019103,-0.45,25,66,29,1.8,29.586575,19.3,15.47,-,10,1.679,0.42583,0.41375,27.1,836,860
4,Communication Services,Telecom Services,TMUS,"T-Mobile US, Inc.",1994,2019,156,129.91,-0.171862,179.43,0.143813,235.0,28,0.53,8,0.001077,4.0,176,18,8,1.9,24.284603,15.7,10.62,0.0,10,3.55,0.34773,0.19468,8.9,289,180
5,Consumer Cyclical,Apparel Retail,TJX,"TJX Companies, Inc. (The)",1987,1985,89,91.00,0.011673,101.90,0.132852,110.0,21,0.88,9,0.001519,1.31,183,6,3,1.8,25.223164,21.7,17.58,0.01,7,0.132,0.11960,0.11670,4.9,109,101
6,Consumer Cyclical,Auto Parts,APTV,Aptiv PLC,1994,2012,88,69.00,-0.223410,114.42,0.287788,155.0,24,1.97,5,-0.000458,1.95,63,2,1,1.9,10.859278,14.8,11.31,-,8,4.485,0.13484,0.09660,3.8,30,24
7,Consumer Cyclical,Auto Parts,LKQ,LKQ Corporation,1998,2016,46,55.00,0.179245,62.75,0.345412,68.0,8,1.29,4,-0.001155,3.07,94,1,0,1.7,13.053221,11.2,10.34,0.03,5,-0.189,0.12883,0.09389,5.7,17,12
8,Consumer Cyclical,Internet Retail,AMZN,"Amazon.com, Inc.",1994,2005,147,128.00,-0.131732,174.52,0.183829,230.0,48,1.17,109,0.007360,1.38,90,71,36,1.7,78.518326,42.1,22.33,-,10,2.357,0.13353,0.07819,1.7,1651,1549
9,Consumer Cyclical,Packaging & Containers,AVY,Avery Dennison Corporation,1990,1987,198,185.00,-0.070118,202.00,0.015330,225.0,11,0.93,0,0.001273,2.64,160,0,0,1.7,33.352448,20.9,16.34,0.02,7,-0.367,0.14046,0.12067,2.3,18,15


In [53]:
GSHEET_NAME = 'SP 500 + Good Opportunites'
TAB_NAME = 'GO' 

def write_df_to_sheet(GSHEET_NAME, TAB_NAME, df):
    gc= gspread.service_account(filename ='sp500opp-81786021999a.json')
    sh=gc.open(GSHEET_NAME)
    worksheet = sh.worksheet(TAB_NAME)
    set_with_dataframe(worksheet, df)

write_df_to_sheet(GSHEET_NAME, TAB_NAME, good_opportunities)  

# Analysys of Divided Stocks based on 5 Year Average Divided Yield, Trailing Dividend Yield and Dividend Yield

In [ ]:
n_largest=5
dividednd_analysis=final_df[['Symbol','Company_Name','Market_Cap','Sector','5Y_Avg_Div_Y','T_An_Div_Y','Div_Y','Rec_Mean','Last_Div_Date','Ex_Div_Date','Beta','52W_Change','ROE','Ent_to_EBITDA','Founded']]
dividednd_analysis=pd.DataFrame(dividednd_analysis)
dividednd_analysis['Type'] = '5Y_Avg_Div_Y'
dividednd_analysis=dividednd_analysis.nlargest(n_largest, '5Y_Avg_Div_Y')
dividednd_analysis2=final_df[['Symbol','Company_Name','Market_Cap','Sector','5Y_Avg_Div_Y','T_An_Div_Y','Div_Y','Rec_Mean','Last_Div_Date','Ex_Div_Date','Beta','52W_Change','ROE','Ent_to_EBITDA','Founded']]
dividednd_analysis2=pd.DataFrame(dividednd_analysis2)
dividednd_analysis2['Type'] = 'T_An_Div_Y'
dividednd_analysis2=dividednd_analysis2.nlargest(n_largest, 'T_An_Div_Y')
dividednd_analysis3=final_df[['Symbol','Company_Name','Market_Cap','Sector','5Y_Avg_Div_Y','T_An_Div_Y','Div_Y','Rec_Mean','Last_Div_Date','Ex_Div_Date','Beta','52W_Change','ROE','Ent_to_EBITDA','Founded']]
dividednd_analysis3=pd.DataFrame(dividednd_analysis3)
dividednd_analysis3['Type'] = 'Div_Y'
dividednd_analysis3=dividednd_analysis3.nlargest(n_largest, 'Div_Y')

colletion = [dividednd_analysis, dividednd_analysis2, dividednd_analysis3]
colletion = pd.concat(colletion)
avg_rec = final_df[['Rec_Mean','Sector']].groupby(['Sector']).mean().reset_index().round(1).sort_values(by=['Rec_Mean'], ascending=False)
colletion=pd.merge(colletion, avg_rec, on='Sector', how='outer', indicator=True)
colletion = colletion[colletion['Company_Name'].notna()].drop(columns=['_merge'])
colletion.rename(columns = {'Rec_Mean_x':'Rec_Mean','Rec_Mean_y':'Rec_Mean_Sector'}, inplace = True)

dividends = pd.pivot_table(colletion, index=['Type','Symbol','Company_Name','Market_Cap','Sector','5Y_Avg_Div_Y','T_An_Div_Y','Div_Y','Rec_Mean','Beta','Last_Div_Date','Ex_Div_Date','52W_Change','ROE','Ent_to_EBITDA','Founded'], values="Market_Cap").reset_index()
dividends=dividends.style.format({ ('5Y_Avg_Div_Y'):'{0:,.1%}',('T_An_Div_Y'):'{0:,.1%}',('Div_Y'):'{0:,.1%}',('52W_Change'):'{0:,.1%}',('ROE'):'{0:,.1%}',('Rec_Mean'):'{0:,.1f}',('Beta'):'{0:,.1f}',('Ent_to_EBITDA'):'{0:,.0f}',('Market_Cap'):'{0:,.0f}',('Rec_Mean_Sector'):'{0:,.2f}'})

In [ ]:
GSHEET_NAME = 'SP 500 + Good Opportunites'
TAB_NAME = 'Div' 

def write_df_to_sheet(GSHEET_NAME, TAB_NAME, df):
    gc= gspread.service_account(filename ='sp500opp-81786021999a.json')
    sh=gc.open(GSHEET_NAME)
    worksheet = sh.worksheet(TAB_NAME)
    set_with_dataframe(worksheet, df)

write_df_to_sheet(GSHEET_NAME, TAB_NAME, dividends)  

AttributeError: 'Styler' object has no attribute 'shape'

In [ ]:
import pandas as pd

# Creating a DataFrame with sample values
data = {'Value': [-0.043424, 0.123456, 0.789012]}
df = pd.DataFrame(data)

# Formatting the 'Value' column to display as percentages
styled_df = df.style.format({'Value': '{:.2%}'})

# Displaying the formatted DataFrame
styled_df

,Value
0,-4.34%
1,12.35%
2,78.90%
